In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from refnx.reflect import SLD, Structure, ReflectModel
from refnx.dataset import ReflectDataset
from tof_simulator import ReflectSimulator
from parabolic_brush import ParabolicBrush

In [ ]:
si = SLD(2.07)
sio2 = SLD(3.47)
dtol = SLD(5.6)
# same brush as that in Karim1994.
brush = ParabolicBrush(1.4, 0.175, 95, 1.0, 150, 10, microslab_max_thickness=3)

In [ ]:
s = si | sio2(15, 3) | brush | dtol(0, 3)
s.contract = 1
model = ReflectModel(s, bkg=5e-7)

In [ ]:
a0 = ReflectSimulator(model, 0.8, rebin=2.5)
a1 = ReflectSimulator(model, 3.5, rebin=2.5)

In [ ]:
a0.run(7000000)
for i in range(500):
    a1.run(1000000)

In [ ]:
d0 = a0.reflectivity
d1 = a1.reflectivity
d0 += d1

In [ ]:
d0 = ReflectDataset('karim_simulation.dat')
d0.plot()
plt.yscale('log')
plt.xscale('log')

In [ ]:
plt.plot(d0.x, d0.y)
plt.plot(d0.x, model(d0.x, x_err=d0.x_err))
plt.yscale('log')
plt.xscale('log')
# plt.xlim(0.01, 0.07)

In [ ]:
d0.save('karim_simulation.dat')